# Sentiment Analysis

Sentiment analysis is also known as opinion mining. Sentiment analysis is a type of text mining that finds and extracts subjective information from source material, assisting businesses in determining the social sentiment associated with their brand, product, or service while monitoring online discussions.

In [2]:
#installing dependency pyspark and findspark
#use the below 2 lines only if the packages are not installed
!pip install pyspark
!pip install findspark

#importing the package pyspark and findspark
import pyspark
import findspark

#initializing findspark to set the environment for pyspark
findspark.init()
findspark.find()

     |████████████████████████████████| 281.4 MB 32 kB/s 
     |████████████████████████████████| 198 kB 49.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=956514581b98ae0012535f854e697d04f9d58f930fe4f196bc3fcb98b074ee6e
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


'/usr/local/lib/python3.7/dist-packages/pyspark'

Starting the Spark Session

In [3]:
from pyspark.sql import SparkSession

#The entry point to programming Spark with the Dataset and DataFrame API. 
spark = SparkSession.builder.appName("NLPSAA").getOrCreate()

Import Important modules required 

In [4]:
#importing pyspark ml sql features
from pyspark.ml import Pipeline 
from pyspark.ml.feature import CountVectorizer,StringIndexer, RegexTokenizer,StopWordsRemover
from pyspark.sql.functions import col, udf,regexp_replace,isnull
from pyspark.sql.types import StringType,IntegerType
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

Now we are loading the dataset. The dataset used here contains the tweets with the sentiment value. The 0 represent negative sentiments and 1 represent positive sentiments.

In [ ]:
#read the csv containing twitter data
news_data = spark.read.csv('training.1600000.processed.noemoticon.csv',header= False)
#printing the data
news_data.printSchema()
news_data.show()
news_data = news_data.limit(500)
news_data.cache

AnalysisException: ignored

We can check the count of totalitems in the dataset for analysis

In [ ]:
#count the rows of dataset
news_data.count()

500

We are selecting the titles of tweets and the corresponding category of each tweet

In [ ]:
#pick only _c5 and _c0 columns and place in title_category
title_category = news_data.select("_c5","_c0")
title_category.show()

+--------------------+---+
|                 _c5|_c0|
+--------------------+---+
|@switchfoot http:...|  0|
|is upset that he ...|  0|
|@Kenichan I dived...|  0|
|my whole body fee...|  0|
|@nationwideclass ...|  0|
|@Kwesidei not the...|  0|
|         Need a hug |  0|
|@LOLTrish hey  lo...|  0|
|@Tatiana_K nope t...|  0|
|@twittera que me ...|  0|
|spring break in p...|  0|
|I just re-pierced...|  0|
|@caregiving I cou...|  0|
|@octolinz16 It it...|  0|
|@smarrison i woul...|  0|
|@iamjazzyfizzle I...|  0|
|Hollis' death sce...|  0|
|about to file taxes |  0|
|@LettyA ahh ive a...|  0|
|@FakerPattyPattz ...|  0|
+--------------------+---+
only showing top 20 rows



This is the custom function definition to count the null values

In [ ]:
#function to count null values in the columns
def null_value_count(df):
  null_columns_counts = [] #initialize array to null
  numRows = df.count() #count the number of rows
  for k in df.columns:
    nullRows = df.where(col(k).isNull()).count() #count null rows
    if(nullRows > 0):
      temp = k,nullRows
      null_columns_counts.append(temp)
  return(null_columns_counts) #return count of null collumns

We are applying the custom function to the data frsme title_category

In [ ]:
null_columns_count_list = null_value_count(title_category)
#spark.createDataFrame(null_columns_count_list, ['Column_With_Null_Value', 'Null_Values_Count']).show()

# Cleaning the dataset

Now we can drop the null values

In [ ]:
#drop not applicable and null values from the category
title_category = title_category.dropna()
title_category.count()
title_category.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------+---+
|_c5                                                                                                                  |_c0|
+---------------------------------------------------------------------------------------------------------------------+---+
|@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D  |0  |
|is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!      |0  |
|@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds                            |0  |
|my whole body feels itchy and like its on fire                                                                       |0  |
|@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there.       |0  |
|@Kwesid

In [ ]:
from functools import reduce

#data containing from the csv
oldColumns = title_category.schema.names
#creating new columns with heading 'Tweets' and 'Sentiment'
newColumns = ['Tweets','Sentiment']

#applying lambda function to create a copy of old columns to new columns
title_category = reduce(lambda title_category, idx: title_category.withColumnRenamed(oldColumns[idx], newColumns[idx]),range(len(oldColumns)), title_category)
title_category.printSchema()
title_category.show()

root
 |-- Tweets: string (nullable = true)
 |-- Sentiment: string (nullable = true)

+--------------------+---------+
|              Tweets|Sentiment|
+--------------------+---------+
|@switchfoot http:...|        0|
|is upset that he ...|        0|
|@Kenichan I dived...|        0|
|my whole body fee...|        0|
|@nationwideclass ...|        0|
|@Kwesidei not the...|        0|
|         Need a hug |        0|
|@LOLTrish hey  lo...|        0|
|@Tatiana_K nope t...|        0|
|@twittera que me ...|        0|
|spring break in p...|        0|
|I just re-pierced...|        0|
|@caregiving I cou...|        0|
|@octolinz16 It it...|        0|
|@smarrison i woul...|        0|
|@iamjazzyfizzle I...|        0|
|Hollis' death sce...|        0|
|about to file taxes |        0|
|@LettyA ahh ive a...|        0|
|@FakerPattyPattz ...|        0|
+--------------------+---------+
only showing top 20 rows



Now we can remove the numbers in tweets

In [ ]:
#cleaning the numbers from tweets
title_category = title_category.withColumn("only_str",regexp_replace(col('Tweets'), '\d+', ''))
title_category.select("Tweets","only_str").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------+
|Tweets                                                                                                               |only_str                                                                                                             |
+---------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------+
|@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D  |@switchfoot http://twitpic.com/yzl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D    |
|is upset that he can't update his Facebook by t

Split the text into constituent words

In [ ]:
#split the text to words or tokens
#https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.ml.feature.RegexTokenizer.html
regex_tokenizer = RegexTokenizer(inputCol="only_str", outputCol="words", pattern="\\W")
raw_words = regex_tokenizer.transform(title_category)
raw_words.show()

+--------------------+---------+--------------------+--------------------+
|              Tweets|Sentiment|            only_str|               words|
+--------------------+---------+--------------------+--------------------+
|@switchfoot http:...|        0|@switchfoot http:...|[switchfoot, http...|
|is upset that he ...|        0|is upset that he ...|[is, upset, that,...|
|@Kenichan I dived...|        0|@Kenichan I dived...|[kenichan, i, div...|
|my whole body fee...|        0|my whole body fee...|[my, whole, body,...|
|@nationwideclass ...|        0|@nationwideclass ...|[nationwideclass,...|
|@Kwesidei not the...|        0|@Kwesidei not the...|[kwesidei, not, t...|
|         Need a hug |        0|         Need a hug |      [need, a, hug]|
|@LOLTrish hey  lo...|        0|@LOLTrish hey  lo...|[loltrish, hey, l...|
|@Tatiana_K nope t...|        0|@Tatiana_K nope t...|[tatiana_k, nope,...|
|@twittera que me ...|        0|@twittera que me ...|[twittera, que, m...|
|spring break in p...|   

Remove the stop words from segregated list of words

In [ ]:
#Removing the stop words from the list of words
#https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.StopWordsRemover.html
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
words_df = remover.transform(raw_words)
words_df.select("words","filtered").show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------+
|words                                                                                                                                |filtered                                                                                     |
+-------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------+
|[switchfoot, http, twitpic, com, yzl, awww, that, s, a, bummer, you, shoulda, got, david, carr, of, third, day, to, do, it, d]       |[switchfoot, http, twitpic, com, yzl, awww, bummer, shoulda, got, david, carr, third, day, d]|
|[is, upset, that, he, can, t, update, his, facebook, by, texting, it, and, migh

Convert text into vectors of token counts

# Partition the dataset into training and test datasets


In [ ]:
#Partition the dataset into trainingData 80% and testData 20%
(trainingData, testData) = words_df.randomSplit([0.8, 0.2],seed = 11)
trainingData.show()
testData.show()

+--------------------+---------+--------------------+--------------------+--------------------+
|              Tweets|Sentiment|            only_str|               words|            filtered|
+--------------------+---------+--------------------+--------------------+--------------------+
| Body Of Missing ...|        0| Body Of Missing ...|[body, of, missin...|[body, missing, n...|
| wonder if Jon lo...|        0| wonder if Jon lo...|[wonder, if, jon,...|[wonder, jon, los...|
|#3 woke up and wa...|        0|# woke up and was...|[woke, up, and, w...|[woke, accident, ...|
|&quot;On popular ...|        0|&quot;On popular ...|[quot, on, popula...|[quot, popular, m...|
|...and, India mis...|        0|...and, India mis...|[and, india, miss...|[india, missed, t...|
|@AmaNorris wow th...|        0|@AmaNorris wow th...|[amanorris, wow, ...|[amanorris, wow, ...|
|@Appomattox_News ...|        0|@Appomattox_News ...|[appomattox_news,...|[appomattox_news,...|
|@B_Barnett I did ...|        0|@B_Barne

# Model Training and Prediction

In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

#tokenizer = Tokenizer(inputCol="text", outputCol="words")
#HashingTF maps a sequence of terms
#https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.HashingTF.html
hashtf = HashingTF(numFeatures=2**16, inputCol="filtered", outputCol='tf')

#IDF stands for Inverse Document Frequency (Common:Rare)
#https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.mllib.feature.IDF.html
idf = IDF(inputCol='tf', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
#label_stringIdx = StringIndexer(inputCol = "target", outputCol = "label")
pipeline = Pipeline(stages=[hashtf, idf])

pipelineFit = pipeline.fit(trainingData)
train_df = pipelineFit.transform(trainingData)
val_df = pipelineFit.transform(testData)
train_df.show(5)

+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+
|              Tweets|Sentiment|            only_str|               words|            filtered|                  tf|            features|
+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+
| Body Of Missing ...|        0| Body Of Missing ...|[body, of, missin...|[body, missing, n...|(65536,[731,3159,...|(65536,[731,3159,...|
| wonder if Jon lo...|        0| wonder if Jon lo...|[wonder, if, jon,...|[wonder, jon, los...|(65536,[19153,329...|(65536,[19153,329...|
|#3 woke up and wa...|        0|# woke up and was...|[woke, up, and, w...|[woke, accident, ...|(65536,[5660,7427...|(65536,[5660,7427...|
|&quot;On popular ...|        0|&quot;On popular ...|[quot, on, popula...|[quot, popular, m...|(65536,[178,1903,...|(65536,[178,1903,...|
|...and, India mis...|        0|..

In [ ]:
from pyspark.sql.types import IntegerType
train_df=train_df.withColumnRenamed('Sentiment', 'label')
train_df=train_df.withColumn("label",train_df.label.cast('int'))
val_df=val_df.withColumnRenamed('Sentiment', 'label')
val_df=val_df.withColumn("label",val_df.label.cast('int'))

## Logistic Regression

In [ ]:
from pyspark.ml.classification import LogisticRegression
## Fitting the model
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
lrModel = lr.fit(train_df)
lrPreds = lrModel.transform(val_df)

In [ ]:
## Evaluating the model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
lr_accuracy = evaluator.evaluate(lrPreds)
print("Accuracy of Logistic Regression is = %g"% (lr_accuracy))

Accuracy of Logistic Regression is = 1


## Decision Tree Model

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
## Fitting the model
#https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.DecisionTreeClassifier.html
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 3)
dtModel = dt.fit(train_df)
dtPreds = dtModel.transform(val_df)

In [ ]:
## Evaluating the model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
dt_accuracy = evaluator.evaluate(dtPreds)
#Accuracy of Decision Tree
print("Accuracy of Decision Trees is = %g"% (dt_accuracy))


Accuracy of Decision Trees is = 1


# Naive Bayes Model

In [ ]:
#applying NaiveBays algorithm
nb = NaiveBayes(modelType="multinomial",labelCol="label", featuresCol="features")
nbModel = nb.fit(train_df)
#get the prediction by transforming the model
nb_predictions = nbModel.transform(val_df)

In [ ]:
nb_predictions.select("prediction", "label", "features").show(100)

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    0|(65536,[2464,1430...|
|       0.0|    0|(65536,[1903,4529...|
|       0.0|    0|(65536,[1903,9859...|
|       0.0|    0|(65536,[12806,230...|
|       0.0|    0|(65536,[14013,171...|
|       0.0|    0|(65536,[22076,344...|
|       0.0|    0|(65536,[22351,260...|
|       0.0|    0|(65536,[8741,2418...|
|       0.0|    0|(65536,[13712,159...|
|       0.0|    0|(65536,[2762,6040...|
|       0.0|    0|(65536,[5827,8449...|
|       0.0|    0|(65536,[7173,3399...|
|       0.0|    0|(65536,[2548,2888...|
|       0.0|    0|(65536,[31448,478...|
|       0.0|    0|(65536,[308,13889...|
|       0.0|    0|(65536,[6040,6122...|
|       0.0|    0|(65536,[1198,4207...|
|       0.0|    0|(65536,[2338,4166...|
|       0.0|    0|(65536,[3386,2202...|
|       0.0|    0|(65536,[9859,1298...|
|       0.0|    0|(65536,[7194,1908...|
|       0.0|    0|(65536,[6042,8923...|


In [ ]:
#evaluate and print the accuracy
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
nb_accuracy = evaluator.evaluate(nb_predictions)
print("Accuracy of NaiveBayes is = %g"% (nb_accuracy))
print("Test Error of NaiveBayes = %g " % (1.0 - nb_accuracy))

Accuracy of NaiveBayes is = 1
Test Error of NaiveBayes = 0 


In [ ]:
#https://www.datatechnotes.com/2021/12/mllib-naive-bayes-classification.html